In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import random
import seaborn as sns
from scipy import stats as stats

# Part 1
## kNN using dataset on heart disease obtained from https://archive.ics.uci.edu/ml/datasets/Heart+Disease
#### Data attributes:
* age: age in years 
* sex: sex (1 = male; 0 = female) 
* cp: chest pain type 
    - Value 1: typical angina 
    - Value 2: atypical angina 
    - Value 3: non-anginal pain 
    - Value 4: asymptomatic 
* trestbps: resting blood pressure (in mm Hg on admission to the hospital) 
* chol: serum cholestoral in mg/dl 
* fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) 
* restecg: resting electrocardiographic results 
    - Value 0: normal 
    - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV) 
    - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria 
* thalach: maximum heart rate achieved 
* exang: exercise induced angina (1 = yes; 0 = no) 
* oldpeak = ST depression induced by exercise relative to rest 
* slope: the slope of the peak exercise ST segment 
    - Value 1: upsloping 
    - Value 2: flat 
    - Value 3: downsloping 
* ca: number of major vessels (0-3) colored by flourosopy 
* thal: 3 = normal; 6 = fixed defect; 7 = reversable defect 
* num: diagnosis of heart disease (angiographic disease status) 
    - Value 0: absence.
    - Value 1,2,3,4: presence of heart disease

### Create dataframe and modify num column to reflect presence of disease (1) or no presence of disase (0)

In [ ]:
df = pd.read_csv('./cleveland.csv')
df = df.rename({'num':'disease'}, axis=1)

#convert disease column in to boolean values that 0 means no disease and >0 means has disease
df['disease'] = df.disease.apply(lambda x: min(x, 1))
display(df.head(5))

In [ ]:
# standardize the data for set of attributes
df['age_s'] = (df.age-df.age.mean())/df.age.std()
df['trestbps_s'] = (df.trestbps-df.trestbps.mean())/df.trestbps.std()
df['chol_s'] = (df.chol-df.chol.mean())/df.chol.std()
df['thalach_s'] = (df.thalach-df.thalach.mean())/df.thalach.std()
df['oldpeak_s'] = (df.oldpeak-df.oldpeak.mean())/df.oldpeak.std()

#data cleaning
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)
# We'll also convert these types to floats since the '?' forced them to be strings
df=df.apply(pd.to_numeric)

# #general standardization
# numeric_columns = df.select_dtypes(include=np.number).columns
# df[numeric_columns] = (df[numeric_columns] - df[numeric_columns].mean()) / df[numeric_columns].std()

display(df.head())

### optimal attribute selection with GirdSearchCV

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

#modify the standardized dataset for implementation
df1=df[['age_s','trestbps_s','chol_s','thalach_s','oldpeak_s','disease']] #,'sex','cp','fbs','restecg','slope','ca','thal'

X1=df1.drop('disease',axis=1)
#display(X1.head())
y1=df1['disease']
#display(y1.head())

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.2, random_state = 42)

rfc = RandomForestClassifier()
forest_params = [{'max_depth': list(range(5, 11)), 'max_features': list(range(0,6))}]
clf = GridSearchCV(rfc, forest_params, cv = 10, scoring='accuracy')
clf.fit(X1_train, y1_train)
print(clf.best_params_)
print(clf.best_score_)

### optimal value of k

In [ ]:
def get_scores(k):
  # Use knn on age. First create a nearest neighbors object.
  nn = NearestNeighbors(n_neighbors=k, metric='euclidean', algorithm='auto')

  # This builds an index data structure under the hood for query performance
  X = df[['age_s', 'trestbps_s','chol_s','thalach_s','oldpeak_s','disease']].values
  fit = nn.fit(X)

  # Get random patients to test on
  n = 50
  patients = df.sample(n)
  patientsX = patients[['age_s','trestbps_s','chol_s','thalach_s','oldpeak_s','disease']].values
  patientsy = patients[['disease']].values
  # display(patients)

  # Find the k nearest neighbors to the patient.
  distances, indices = fit.kneighbors(patientsX)
  # print('indices of k-nearest neighbors for each patient:')
  # display(indices)

  y_pred = []
  for i in range(n):
      # print('nearest neighbors to patient: {}:'.format(patientsX[i]))
      nbrs = df.iloc[indices[i]]
      # Drop the patient of interest
      nbrs = nbrs.drop(patients.index[i], errors='ignore')
      # display(nbrs)

      healthy = nbrs[nbrs.disease == 0].count().disease
      sick = nbrs[nbrs.disease == 1].count().disease
      predict = 0 if (healthy > sick) else 1
      # print(f'healthy: {healthy}, sick: {sick}, predicted: {predict}, actual: {patientsy[i][0]}')
      y_pred.append(predict)

  # This is where we would compile how many patients are predicted
  # correctly. Remember:
  #    precision = tp/(tp+fp)  ("sloppiness")
  #    recall    = tp/(tp+fn)  ("What percentage did we find?")
  #    f-score - a balance between precision and recall
  #    support - number of positive labels
  return precision_recall_fscore_support(patientsy, y_pred, labels=[1])

kvals = range(2, 40, 2)
scores = [get_scores(k) for k in kvals]
# print(scores)

scores = [(p[0], r[0], f[0], s[0]) for (p,r,f,s) in scores]
scores = list(zip(*scores))

plt.plot(kvals, scores[2],marker="o",label='f_score')
plt.plot(kvals, scores[0],marker="o",label='precision')
plt.plot(kvals, scores[1],marker="o",label='recall')
plt.xlabel('k-values')
plt.ylabel('scores')
plt.legend()
plt.title('plot for precision,recall and f_scores')

##### It could be seen that k=13 has the highest f-score which has better recall and precision values

### monte carlo simulation & cross validation

In [ ]:
from sklearn import neighbors
from sklearn.model_selection import train_test_split
import random

def monte_carlo_cross_validation(df: pd.DataFrame, label: str, 
                                 model: NearestNeighbors, 
                                 iterations: int,
                                 features: list = None, 
                                 min_test_split: float= 0.15, 
                                 max_test_split: float = 0.35) -> dict:
  # https://towardsdatascience.com/cross-validation-k-fold-vs-monte-carlo-e54df2fc179b#9a83
  # https://stats.stackexchange.com/a/60967
  assert 0 < min_test_split < 1, "min_test_split must be between 0 and 1!"
  assert 0 < max_test_split < 1, "max_test_split must be between 0 and 1!"

  # Correct any issue with the list of features
  if features is None:
    features = [c for c in df.columns if c != label]
  elif not isinstance(features, list):
    features = [features]

  # Can't use rows with NaN
  df = df.dropna()

  all_features = df[features].values
  all_labels = df[label].values

  results = {
      "precision": 0,
      "recall": 0,
      "f-score": 0,
      "support": 0
  }
  dfAllScores = pd.DataFrame(columns=results.keys())
  # Perform the validation
  for _ in range(iterations):
    # Create the split
    test_split = random.uniform(min_test_split, max_test_split)
    features_train, features_test, labels_train, labels_test = \
      train_test_split(all_features, all_labels, test_size=test_split)
    
    # Fit the model
    fit = model.fit(features_train)

    # Evaluate on the test set
    distances, indices = fit.kneighbors(features_test)

    # Get the predictions
    predictions = []
    for i in range(len(features_test)):
      neighbor_indices = indices[i]
      neighbor_labels = labels_train[neighbor_indices]
      unique, counts = np.unique(neighbor_labels, return_counts=True)
      # neighbor_counts = dict(zip(unique, counts))
      # has_not = neighbor_counts[False] if False in neighbor_counts else 0
      # has = neighbor_counts[True] if True in neighbor_counts else 0
      # predictions.append(0 if (has_not > has) else 1)
      prediction = max(zip(unique, counts), key=lambda t: t[1])[0]
      predictions.append(prediction)

    # Calculate the scores
    # TODO: I think I could do this after all iterations using a 2D array
    p, r, f, s = precision_recall_fscore_support(labels_test, predictions, labels=[1])
    
    # Update the results
    dfAllScores.loc[len(dfAllScores)] = {
      "precision": p[0],
      "recall": r[0],
      "f-score": f[0],
      "support": s[0]
    }
  # Average the results
  results["precision"] = dfAllScores["precision"].mean()
  results["recall"] = dfAllScores["recall"].mean()
  results["f-score"] = dfAllScores["f-score"].mean()
  results["support"] = dfAllScores["support"].mean()
  return results, dfAllScores

### give range for k, number of k-fold cross validation, get scores and plot optimal k-value

In [ ]:
dfKResults = pd.DataFrame(columns=["k", "precision", "recall", "f-score", "support"])
dfKResults.set_index('k', inplace=True)
dfKAllScores = pd.DataFrame(columns=["k", "precision", "recall", "f-score", "support"])

for k in range(2, 21,1):
  nn = NearestNeighbors(n_neighbors=k, metric='euclidean', algorithm='auto')
  results, dfAllScores = monte_carlo_cross_validation(df, "disease", nn, iterations=10,min_test_split=0.20, max_test_split=0.30)
  dfAllScores['k'] = k
  dfKResults.loc[k] = results
  dfKAllScores = pd.concat([dfKAllScores, dfAllScores])

dfKResults.reset_index(inplace=True)
dfKAllScores.set_index('k', inplace=True)
print("Averages")
display(dfKResults)

ax1=sns.lineplot(x='k', y='recall', data=dfKResults, marker="o", label="recall");
ax2=sns.lineplot(x='k', y='precision', data=dfKResults, marker='o', label="precision");
ax1.lines[0].set_linestyle("--")
ax2.lines[1].set_linestyle("--")
ax = sns.lineplot(x='k', y='f-score', data=dfKResults, marker='o', label="f-score");
#ax.set_ylim(0.4, 0.7);
ax.set_xlabel('k_values')
ax.set_ylabel('scores')
ax.set_title('lineplot for precision,recall & f_scores')

##### It could be seen that k=13 has the highest f-score which has better recall and precision values

# Part 2
## kNN using dataset on diabetes 
#### Dataset obtained from https://www.kaggle.com/datasets/houcembenmansour/predict-diabetes-based-on-diagnostic-measures

### Step 1: Create dataframe 
 * ### Transform gender and diabetes columns to numeric representation (0 or 1)
 * ### Transform chol_hdl_ratio, bmi, and waist_hip_ratio columns to float representations (instead of comma seperated numbers)
 * ### Drop patient_number column (essentially a second index)

In [ ]:
diabetes_df = pd.read_csv('./diabetes.csv')

diabetes_df['diabetes'] = diabetes_df.diabetes.apply(lambda x: 0 if x=='No diabetes' else 1)
diabetes_df['gender'] = diabetes_df.gender.apply(lambda x: 0 if x=='female' else 1)
diabetes_df['chol_hdl_ratio'] = diabetes_df.chol_hdl_ratio.apply(lambda x: float(str(x).replace(',', '.')))
diabetes_df['bmi'] = diabetes_df.bmi.apply(lambda x: float(str(x).replace(',', '.')))
diabetes_df['waist_hip_ratio'] = diabetes_df.waist_hip_ratio.apply(lambda x: float(str(x).replace(',', '.')))
diabetes_df.drop(columns='patient_number', inplace=True)

diabetes_df.head()

### Step 2: Make standardized versions of each column in the dataframe

In [ ]:
diabetes_df['cholesterol_s'] = (diabetes_df.cholesterol-diabetes_df.cholesterol.mean())/diabetes_df.cholesterol.std()
diabetes_df['glucose_s'] = (diabetes_df.glucose-diabetes_df.glucose.mean())/diabetes_df.glucose.std()
diabetes_df['hdl_chol_s'] = (diabetes_df.hdl_chol-diabetes_df.hdl_chol.mean())/diabetes_df.hdl_chol.std()
diabetes_df['chol_hdl_ratio_s'] = (diabetes_df.chol_hdl_ratio-diabetes_df.chol_hdl_ratio.mean())/diabetes_df.chol_hdl_ratio.std()
diabetes_df['age_s'] = (diabetes_df.age-diabetes_df.age.mean())/diabetes_df.age.std()
diabetes_df['gender_s'] = (diabetes_df.gender-diabetes_df.gender.mean())/diabetes_df.gender.std()
diabetes_df['height_s'] = (diabetes_df.height-diabetes_df.height.mean())/diabetes_df.height.std()
diabetes_df['weight_s'] = (diabetes_df.weight-diabetes_df.weight.mean())/diabetes_df.weight.std()
diabetes_df['bmi_s'] = (diabetes_df.bmi-diabetes_df.bmi.mean())/diabetes_df.bmi.std()
diabetes_df['systolic_bp_s'] = (diabetes_df.systolic_bp-diabetes_df.systolic_bp.mean())/diabetes_df.systolic_bp.std()
diabetes_df['diastolic_bp_s'] = (diabetes_df.diastolic_bp-diabetes_df.diastolic_bp.mean())/diabetes_df.diastolic_bp.std()
diabetes_df['waist_s'] = (diabetes_df.waist-diabetes_df.waist.mean())/diabetes_df.waist.std()
diabetes_df['hip_s'] = (diabetes_df.hip-diabetes_df.hip.mean())/diabetes_df.hip.std()
diabetes_df['waist_hip_ratio_s'] = (diabetes_df.waist_hip_ratio-diabetes_df.waist_hip_ratio.mean())/diabetes_df.waist_hip_ratio.std()

diabetes_df.head()

### Step 3: Create function get_scores() 
 * ### takes a k value as input
 * ### builds a kNN model
 * ### returns the recall, precision, and f-score results 

In [ ]:
def get_scores(k):
    nn = NearestNeighbors(n_neighbors=k, metric='euclidean', algorithm='auto')

    X = diabetes_df[['cholesterol_s', 'glucose_s', 'chol_hdl_ratio_s', 'bmi_s', 'systolic_bp_s', 'diastolic_bp_s', 'waist_hip_ratio_s']].values
    y = diabetes_df[['diabetes']].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # X_train is the training data set.
    # y_train is the set of labels to all the data in x_train.
    # X_test is the test data set.
    # y_test is the set of labels to all the data in x_test.
    
    # print(X_train)

    fit = nn.fit(X_train)

    distances, indices = fit.kneighbors(X_test)

    predictions = []
    for i in range(len(X_test)):
        # print('patient: ', X_test[i])
        # print('paintent_d: ', y_test[i])
        # print('indices: ', indices[i])
        # nbrs_g = [X_train[index] for index in indices[i]]
        # print('nbrs_g: ', nbrs_g)
        nbrs_diabetes = [y_train[index] for index in indices[i]]
        nbrs_diabetes = [x[0] for x in nbrs_diabetes]
        # print('nbrs: ', nbrs)

        diabetes = nbrs_diabetes.count(1)
        # print('yes: ', diabetes)
        no_diabetes = nbrs_diabetes.count(0)
        # print('no: ', no_diabetes)

        prediction = 0 if (no_diabetes > diabetes) else 1
        predictions.append(prediction)
    # return
    return precision_recall_fscore_support(y_test, predictions, labels=[1])



### Step 4: Find the optimum k value

In [ ]:
k_values = range(5, 30)
scores = []

for k in k_values:
    k_scores = []
    for i in range(10):
        k_scores.append(get_scores(k)[2][0])
    scores.append(np.array(k_scores).mean())

plt.figure()
plt.title('k value vs f-score')
plt.ylabel('f-score')
plt.xlabel('k value')
plt.plot(k_values, scores)
plt.show()

### Step 5: Build 10 kNN models with a k value of 8 and report results

In [ ]:
results = []
for i in range(10):
    scores = get_scores(8)
    results.append(scores)

f_scores = [result[2][0] for result in results]
mean_f = np.array(f_scores).mean()

print('Mean f-score of all models: ', mean_f, '\n')

print('Individual Model Scores: ')
for i in range(10):
    print('\tModel', i+1, ': precision =', results[i][0][0], 'recall =', results[i][1][0], 'f-score =', results[i][2][0])